# Compute i coefficients

In [1]:
import brightway2 as bw
import pandas as pd
import os

# Set working directry
path = "."
os.chdir(path)

# Local functions
from utils.lookup_func import lookup_geothermal

# Set project
bw.projects.set_current("Geothermal")

# Retrieve activities
wellhead, diesel, steel, cement, water, \
drilling_mud, drill_wst, wells_closr, coll_pipe, \
plant, ORC_fluid, ORC_fluid_wst, diesel_stim, co2,_, _ = lookup_geothermal()
cooling_tower=bw.Database("geothermal energy").search("cooling tower")[0].key

list_act = [wellhead, diesel, steel, cement, water, 
         drilling_mud, drill_wst, wells_closr, coll_pipe,
         plant, cooling_tower, ORC_fluid, ORC_fluid_wst, diesel_stim]

# Retrieve methods 
ILCD_CC = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "climate change total" in str(method)]
ILCD_HH = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "human health" in str(method)]
ILCD_EQ = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "ecosystem quality" in str(method)]
ILCD_RE = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "resources" in str(method)]
ILCD = ILCD_CC + ILCD_HH + ILCD_EQ + ILCD_RE

# Calculate impact of activities
lca = bw.LCA({list_act[0]: 1}, ILCD[0])
lca.lci()
lca.lcia()
coeff = {}
for method in ILCD:
    s=[]
    lca.switch_method(method)
    for act in list_act:
        lca.redo_lcia({act: 1})
        s.append(lca.score)
    coeff[method] = s

# Retrieve CF for co2 emissions    
for method in ILCD:
    CFs = bw.Method(method).load()
    coeff[method].append(next((cf[1] for cf in CFs if cf[0]==co2),0))

# Build matrix
col_names = ["wellhead", "diesel", "steel", "cement", "water", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid", "ORC_fluid_wst", "diesel_stim", "co2"]
coeff_matrix = pd.DataFrame.from_dict(coeff, orient="index", columns=col_names)

# Re-arrange matrix
coeff_matrix["concrete"] = coeff_matrix["cement"] + coeff_matrix["water"] * 1/0.65
coeff_matrix["ORC_fluid_tot"] = coeff_matrix["ORC_fluid"] - coeff_matrix["ORC_fluid_wst"]
coeff_matrix["electricity_stim"] = coeff_matrix["diesel_stim"] * 3.6
coeff_matrix["drill_wst"] = coeff_matrix["drill_wst"] * -1
coeff_matrix=coeff_matrix.drop(columns=["cement", "ORC_fluid", "ORC_fluid_wst", "diesel_stim"])

col_ord = ["wellhead", "diesel", "steel", "concrete", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid_tot", "water", "electricity_stim", "co2"]
coeff_matrix=coeff_matrix[col_ord]

is_= ["i1", "i2.1", "i2.2", "i2.3", "i2.4", "i2.5", \
      "i2.6","i3", "i4.1", "i4.2", "i4.3", "i5.1", "i5.2", "i6"]

if len(coeff_matrix.columns) == len(is_):
    coeff_matrix.columns=is_

# Define symbolic

In [2]:
from sympy import symbols, collect, ratsimp, fraction

In [3]:
# Coefficients
i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6 = \
symbols('i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6')

# Other variables that are actually irrelevant, because they will be multiplied by 0 in the enhanced model
# I wanna keep them though for the sake of completeness
SW_n, S_w, S_el = symbols('SW_n, SW, S_el')

# Main parameters
P_ne, CW_ne, PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP = \
symbols('P_ne, CW_ne, PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP')

# Constants
CT_el, CT_n, W_en, OF,DW = symbols ('CT_el, CT_n, W_E_en, OF, DW')


# Main equation

In [4]:
# Supporting equations for CONVENTIONAL geothermal
W_Pn = P_ne/CW_ne             # production wells
W_In = P_ne/CW_ne / PIratio  # injection wells
W_Mn = W_Pn*D_i*LT            # makeup wells

# Number of wells with success rate
W_n = W_Pn/SR_p + W_In/SR_p + W_Mn/SR_m
#W_n = W_Pn * ( (1 + 1/PIratio)/SR_p + D_i*LT/SR_m)
W_E_en = W_en/SR_e


In [5]:
# Equation
nominator = (W_n + W_E_en) *i1 \
          + W_d* (W_n + W_E_en) * (D*i2_1 + C_S*i2_2 + C_C*i2_3  +  DM*i2_4 + DW*i2_5 + i2_6) + \
          + W_n*CP*i3 \
          + P_ne*i4 \
          + SW_n*S_w * (i5_1 + S_el*i5_2)

c1 = P_ne*CF*(1-AP)*LT*8760000
d1 = CT_el*CT_n*1000*LT
denominator = c1  -  d1

summand = E_co2*i6

eq = nominator/denominator + summand
eq = eq.subs(i4, i4_1 + i4_2*CT_n + i4_3*OF)

In [6]:
# Show equation
eq

E_co2*i6 + (CP*i3*(D_i*LT*P_ne/(CW_ne*SR_m) + P_ne/(CW_ne*SR_p) + P_ne/(CW_ne*PIratio*SR_p)) + P_ne*(CT_n*i4_2 + OF*i4_3 + i4_1) + SW*SW_n*(S_el*i5_2 + i5_1) + W_d*(W_E_en/SR_e + D_i*LT*P_ne/(CW_ne*SR_m) + P_ne/(CW_ne*SR_p) + P_ne/(CW_ne*PIratio*SR_p))*(C_C*i2_3 + C_S*i2_2 + D*i2_1 + DM*i2_4 + DW*i2_5 + i2_6) + i1*(W_E_en/SR_e + D_i*LT*P_ne/(CW_ne*SR_m) + P_ne/(CW_ne*SR_p) + P_ne/(CW_ne*PIratio*SR_p)))/(8760000*CF*LT*P_ne*(1 - AP) - 1000*CT_el*CT_n*LT)

# Get parameters values

In [7]:
from cge_klausen import parameters
parameters.static()    

# Rename parameters (in accordance with equation)

# Parameters of simplified models
CW_ne_val    = parameters["gross_power_per_well"]
E_co2_val    = parameters["co2_emissions"]
D_i_val      = parameters["initial_harmonic_decline_rate"]
SR_p_val     = parameters["success_rate_primary_wells"]/100
W_d_val      = parameters["average_depth_of_wells"]

# Parameters to be fixed
P_ne_val     = parameters["installed_capacity"]
PIratio_val  = parameters["production_to_injection_ratio"]
LT_val       = parameters["lifetime"]
SR_m_val     = parameters["success_rate_makeup_wells"]/100
SR_e_val     = parameters["success_rate_exploration_wells"]/100
D_val        = parameters["specific_diesel_consumption"]
C_S_val      = parameters["specific_steel_consumption"]
C_C_val      = parameters["specific_cement_consumption"]
DM_val       = parameters["specific_drilling_mud_consumption"]
CP_val       = parameters["collection_pipelines"]
CF_val       = parameters["capacity_factor"]
AP_val       = parameters["auxiliary_power"]

# Constants
CT_el_val  = 864
CT_n_val   = 7/303.3
W_en_val   = 3 * 0.3
OF_val     = 0
DW_val     = 450

par_dict = { P_ne: P_ne_val,
             PIratio: PIratio_val,
             LT: LT_val,
             SR_m: SR_m_val,
             SR_e: SR_e_val,
             D: D_val,
             C_S: C_S_val,
             C_C: C_C_val,
             DM: DM_val,
             CP: CP_val,
             CF: CF_val,
             AP: AP_val,
             CT_el: CT_el_val,
             CT_n: CT_n_val,
             W_en: W_en_val,
             OF: OF_val,
             DW: DW_val}

# Groups of environmental categories

In [8]:
group1 = ["climate change total"]

group2 = ["carcinogenic effects", "ionising radiation", "non-carcinogenic effects", 
          "ozone layer depletion", "photochemical ozone creation", "respiratory effects, inorganics",
          "freshwater and terrestrial acidification", "freshwater ecotoxicity", 
          "freshwater eutrophication", "marine eutrophication", "terrestrial eutrophication",
          "minerals and metals", "dissipated water", "fossils", "land use"]

# Simplified equations

In [9]:
# Define symbols
alpha_1, alpha_2, beta_1, beta_2, beta_3, beta_4, beta_5, beta_6 = \
symbols('alpha_1, alpha_2, beta_1, beta_2, beta_3, beta_4, beta_5, beta_6')

## Group 1 (Climate Change)

In [10]:
eq_group1_all_thresholds = alpha_1*E_co2 + alpha_2
eq_group1_all_thresholds

E_co2*alpha_1 + alpha_2

## Group 2 

### Threshold 15/20%

In [11]:
eq_group2_15_20 = (W_d*beta_1 + beta_2)/CW_ne + CW_ne*beta_3 + beta_4
eq_group2_15_20

CW_ne*beta_3 + beta_4 + (W_d*beta_1 + beta_2)/CW_ne

## Threshold 10%

In [12]:
eq_group2_10 = (D_i*W_d*beta_1 + D_i*beta_2 + W_d*beta_3 + beta_4) / (CW_ne) + W_d*beta_5 + beta_6
eq_group2_10 

W_d*beta_5 + beta_6 + (D_i*W_d*beta_1 + D_i*beta_2 + W_d*beta_3 + beta_4)/CW_ne

## Threshold 5%

In [13]:
eq_group2_05 = (D_i*SR_p*W_d*beta_1 + D_i*SR_p*beta_2 + W_d*beta_3 + beta_4) / (CW_ne*SR_p) + W_d*beta_5 + beta_6
eq_group2_05 

W_d*beta_5 + beta_6 + (D_i*SR_p*W_d*beta_1 + D_i*SR_p*beta_2 + W_d*beta_3 + beta_4)/(CW_ne*SR_p)

# Compute alphas 

## All thresholds

In [27]:
eq_alpha

E_co2*i6 + 6.29186220504825e-11*SW*SW_n*(S_el*i5_2 + i5_1) + 3.05119441393473e-9*i1 + 0.0151034123489769*i2_1 + 0.000686518743135314*i2_2 + 0.000274607497254126*i2_3 + 4.46237183037954e-6*i2_4 + 0.00308933434410891*i2_5 + 6.86518743135314e-6*i2_6 + 1.48344864519275e-6*i3 + 4.40430354353378e-9*i4_1 + 1.01648944295208e-10*i4_2

In [14]:
# Replace parameters in equation
eq_alpha = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        D_i: D_i_val,
        CW_ne: CW_ne_val,
        W_d: W_d_val}
eq_alpha = eq_alpha.subs(repl)

alpha_dict = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group1:
        eq_alpha_is = eq_alpha.subs(is_dict)
        alpha1 = collect(eq_alpha_is, E_co2, evaluate=False)[E_co2]
        alpha2 = collect(eq_alpha_is, E_co2, evaluate=False)[1]
        alpha_dict[str(method)] = {'alpha1': alpha1, 'alpha2': alpha2}
           

In [15]:
# Example of the formula
eq_alpha_is

1.0*E_co2 + 0.0047820519310311

In [16]:
alpha_df = pd.DataFrame.from_dict(alpha_dict, orient='index')
alpha_df = alpha_df.astype(float)
alpha_df_5 = alpha_df_10 = alpha_df_15 = alpha_df_20 = alpha_df
alpha_df

,alpha1,alpha2
"('ILCD 2.0 2018 midpoint no LT', 'climate change', 'climate change total')",1.0,0.004782


# Compute betas

## Threshold 15-20%

In [17]:
# Threshold = 15%/20%
eq_beta = eq.subs(par_dict)
eq_beta = eq_beta.subs(E_co2, E_co2_val)
repl = {SR_p: SR_p_val, D_i: D_i_val} 
eq_beta = eq_beta.subs(repl)
        
beta_dict_15 = {}     
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group2:
        eq_beta_is = eq_beta.subs(is_dict)
        eq_beta_is = ratsimp(eq_beta_is)
        temp = collect(eq_beta_is, [W_d,1/CW_ne], evaluate=False)
        beta1 = collect(temp[list(temp.keys())[1]], W_d, evaluate=False)[W_d]
        beta2 = collect(temp[list(temp.keys())[1]], W_d, evaluate=False)[1]
        beta3 = temp[W_d]
        beta4 = temp[1]
        
        beta_dict_15[str(method)] = {'beta1': beta1, 'beta2': beta2,
                                     'beta3': beta3, 'beta4': beta4}


beta_dict_20 = beta_dict_15


In [18]:
# Show example of the formula
eq_beta_is

6.00895196364021e-13*W_d + 3.41730446072861e-8 + 1.0*(1.24520873118428e-10*W_d + 1.98780044501476e-7)/CW_ne

In [19]:
beta_df_15 = pd.DataFrame.from_dict(beta_dict_15, orient='index')
beta_df_15 = beta_df_15.astype(float)

#20% and 15% threshold have the same configuration
beta_df_20 = beta_df_15

beta_df_15

,beta1,beta2,beta3,beta4
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",1.801969e-12,1.254558e-09,8.695687e-15,7.574660e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation')",3.140645e-07,1.570850e-04,1.515568e-09,2.060965e-05
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",4.871968e-12,3.005852e-09,2.351045e-14,2.840649e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion')",1.024558e-12,3.882077e-10,4.944165e-15,7.122354e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation')",8.135510e-08,2.809408e-05,3.925919e-10,2.713246e-06
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",4.104132e-13,5.083959e-10,1.980514e-15,4.521798e-11
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification')",6.937874e-08,3.415079e-05,3.347981e-10,4.413580e-06
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",5.485830e-05,1.593465e-02,2.647274e-07,1.323571e-03
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",3.544440e-10,2.850529e-07,1.710426e-12,3.743753e-08
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'marine eutrophication')",2.614162e-08,6.909745e-06,1.261505e-10,8.334420e-07


## Threshold = 10%

In [20]:
eq_beta_10 = eq.subs(par_dict)
eq_beta_10 = eq_beta_10.subs(E_co2, E_co2_val)
repl = {SR_p: SR_p_val} 
eq_beta_10 = eq_beta_10.subs(repl)
        
beta_dict_10 = {}   
    
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group2:
        eq_beta_10_is = eq_beta_10.subs(is_dict)
        eq_beta_10_is = ratsimp(eq_beta_10_is)
               
        temp   = collect(eq_beta_10_is, [W_d, 1/CW_ne,], evaluate=False)
        temp_2 = collect(temp[list(temp.keys())[1]], [D_i*W_d, D_i, W_d], evaluate=False)
        
        beta1  = temp_2[D_i*W_d]      
        beta2  = temp_2[D_i]
        beta3  = temp_2[W_d]
        beta4  = temp_2[1]
              
        beta5 = temp[W_d]
        beta6 = temp[1]
        
        beta_dict_10[str(method)] = {'beta1': beta1, 'beta2': beta2,
                                     'beta3': beta3, 'beta4': beta4,
                                     'beta5': beta5, 'beta6': beta6}      
    

In [21]:
# Example of the formula
eq_beta_10_is

6.00895196364021e-13*W_d + 3.41730446072861e-8 + 1.0*(1.18466716174128e-9*D_i*W_d + 1.89115435214146e-6*D_i + 6.52875150313635e-11*W_d + 1.04222326894403e-7)/CW_ne

In [22]:
beta_df_10 = pd.DataFrame.from_dict(beta_dict_10, orient='index')
beta_df_10 = beta_df_10.astype(float)
beta_df_10

,beta1,beta2,beta3,beta4,beta5,beta6
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",1.714358e-11,1.193562e-08,9.447901e-13,6.577772e-10,8.695687e-15,7.574660e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation')",2.987948e-06,1.494476e-03,1.646671e-07,8.236120e-05,1.515568e-09,2.060965e-05
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",4.635095e-11,2.859709e-08,2.554421e-12,1.575998e-09,2.351045e-14,2.840649e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion')",9.747440e-12,3.693332e-09,5.371856e-13,2.035411e-10,4.944165e-15,7.122354e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation')",7.739965e-07,2.672816e-04,4.265528e-08,1.473000e-05,3.925919e-10,2.713246e-06
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",3.904591e-12,4.836779e-09,2.151837e-13,2.665570e-10,1.980514e-15,4.521798e-11
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification')",6.600557e-07,3.249039e-04,3.637595e-08,1.790560e-05,3.347981e-10,4.413580e-06
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",5.219111e-04,1.515991e-01,2.876274e-05,8.354692e-03,2.647274e-07,1.323571e-03
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",3.372111e-09,2.711937e-06,1.858385e-10,1.494560e-07,1.710426e-12,3.743753e-08
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'marine eutrophication')",2.487063e-07,6.573796e-05,1.370631e-08,3.622847e-06,1.261505e-10,8.334420e-07


## Threshold = 5%

In [23]:
eq_beta_5 = eq.subs(par_dict)
eq_beta_5 = eq_beta_5.subs(E_co2, E_co2_val)
        
beta_dict_5 = {}       
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group2:
        eq_beta_5_is = eq_beta_5.subs(is_dict)
        eq_beta_5_is = ratsimp(eq_beta_5_is)
               
        temp   = collect(eq_beta_5_is, [W_d, 1/CW_ne*SR_p], evaluate=False)
        temp_2 = collect(temp[list(temp.keys())[1]], [SR_p*W_d*D_i, D_i*SR_p, W_d], evaluate=False)
        
        beta1  = temp_2[D_i*SR_p*W_d]
        beta2  = temp_2[D_i*SR_p]
        beta3  = temp_2[W_d]
        beta4  = temp_2[1]
               
        beta5 = temp[W_d]
        beta6 = temp[1]
        
        beta_dict_5[str(method)] = {'beta1': beta1, 'beta2': beta2,
                                     'beta3': beta3, 'beta4': beta4,
                                     'beta5': beta5, 'beta6': beta6}      
    

In [24]:
# Show example of the formula
eq_beta_5_is

6.00895196364021e-13*W_d + 3.41730446072861e-8 + 1.0*(1.18466716174128e-9*D_i*SR_p*W_d + 1.89115435214146e-6*D_i*SR_p + 4.70927962522877e-11*W_d + 7.51770197260474e-8)/(CW_ne*SR_p)

In [25]:
beta_df_5 = pd.DataFrame.from_dict(beta_dict_5, orient='index')
beta_df_5 = beta_df_5.astype(float)
beta_df_5

,beta1,beta2,beta3,beta4,beta5,beta6
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",1.714358e-11,1.193562e-08,6.814903e-13,4.744639e-10,8.695687e-15,7.574660e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation')",2.987948e-06,1.494476e-03,1.187767e-07,5.940828e-05,1.515568e-09,2.060965e-05
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",4.635095e-11,2.859709e-08,1.842539e-12,1.136789e-09,2.351045e-14,2.840649e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion')",9.747440e-12,3.693332e-09,3.874795e-13,1.468170e-10,4.944165e-15,7.122354e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation')",7.739965e-07,2.672816e-04,3.076785e-08,1.062496e-05,3.925919e-10,2.713246e-06
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",3.904591e-12,4.836779e-09,1.552150e-13,1.922713e-10,1.980514e-15,4.521798e-11
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification')",6.600557e-07,3.249039e-04,2.623848e-08,1.291556e-05,3.347981e-10,4.413580e-06
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",5.219111e-04,1.515991e-01,2.074697e-05,6.026356e-03,2.647274e-07,1.323571e-03
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",3.372111e-09,2.711937e-06,1.340479e-10,1.078047e-07,1.710426e-12,3.743753e-08
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'marine eutrophication')",2.487063e-07,6.573796e-05,9.886552e-09,2.613210e-06,1.261505e-10,8.334420e-07


# Save to excel

In [26]:
with pd.ExcelWriter(os.path.join(path, 'generated_files\Simplified models coefficients conventional - symbolic - thresholds.xlsx')) as writer:
    alpha_df_5.to_excel(writer, sheet_name='alpha_5%')
    alpha_df_10.to_excel(writer, sheet_name='alpha_10%')
    alpha_df_15.to_excel(writer, sheet_name='alpha_15%')
    alpha_df_20.to_excel(writer, sheet_name='alpha_20%')
    beta_df_5.to_excel(writer, sheet_name='beta_5%')
    beta_df_10.to_excel(writer, sheet_name='beta_10%')
    beta_df_15.to_excel(writer, sheet_name='beta_15%')
    beta_df_20.to_excel(writer, sheet_name='beta_20%')